Import packages

In [38]:
import pandas as pd
from cassandra.cluster import Cluster


Connect to the cassandra database

In [39]:
clstr = Cluster()
session = clstr.connect()

In [40]:
session.execute("DROP KEYSPACE IF EXISTS m14")

Checking the existing keyspaces

In [41]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04


Create a keyspace called m14

In [42]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

Create a table in this keyspace

In [43]:

session.execute("""
CREATE TABLE IF NOT EXISTS m14.datawarehouse ( 
    SKU INT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(SKU)   
);
""")

Load Data

In [44]:
df = pd.read_csv('dwh.csv')
df

,SKU,name,description,warehouse_num
0,121,'Apple','iphone 14 is available in our storage from ne...,101
1,122,'Samsung','Samsung laptop sales diminshed in our location',102
2,123,'lg','refrigerator is the new product with AI techn...,103
3,124,'apple','apple laptop needed repairs',101
4,125,'apple','iwatch is available under $500',101
5,126,'samsung','Samsung galaxy s23 sales are peaked this year',102
6,127,'lg','lg is going to stop selling phone. No lg phon...,103


In [45]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

SKU = 121, name =0, description = 'iphone 14 is available in our storage from next week', warehouse_num = 101
SKU = 122, name =1, description = 'Samsung laptop sales diminshed in our location', warehouse_num = 102
SKU = 123, name =2, description = 'refrigerator is the new product with AI technology', warehouse_num = 103
SKU = 124, name =3, description = 'apple laptop needed repairs', warehouse_num = 101
SKU = 125, name =4, description = 'iwatch is available under $500', warehouse_num = 101
SKU = 126, name =5, description = 'Samsung galaxy s23 sales are peaked this year', warehouse_num = 102
SKU = 127, name =6, description = 'lg is going to stop selling phone. No lg phones in our storage from next quarter ', warehouse_num = 103


In [46]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (121, 'Apple', 'iphone 14 is available in our storage from next week', 101);
        

        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (122, 'Samsung', 'Samsung laptop sales diminshed in our location', 102);
        

        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (123, 'lg', 'refrigerator is the new product with AI technology', 103);
        

        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (124, 'apple', 'apple laptop needed repairs', 101);
        

        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (125, 'apple', 'iwatch is available under $500', 101);
        

        INSERT INTO m14.datawarehouse (SKU, name, description, warehouse_num)     
        VALUES (126, 'samsung', 'Samsung galaxy

In [47]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.datawarehouse")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

SKU=123, name=lg, description=refrigerator is the new product with AI technology, warehouse_num=103
SKU=125, name=apple, description=iwatch is available under $500, warehouse_num=101
SKU=122, name=Samsung, description=Samsung laptop sales diminshed in our location, warehouse_num=102
SKU=121, name=Apple, description=iphone 14 is available in our storage from next week, warehouse_num=101
SKU=126, name=samsung, description=Samsung galaxy s23 sales are peaked this year, warehouse_num=102
SKU=127, name=lg, description=lg is going to stop selling phone. No lg phones in our storage from next quarter , warehouse_num=103
SKU=124, name=apple, description=apple laptop needed repairs, warehouse_num=101


Creating an index on warehouse_num

In [48]:
session.execute("""
                CREATE INDEX IF NOT EXISTS ON m14.datawarehouse(warehouse_num);""")

Search for all products located at a given location 

In [49]:

rows = session.execute("select (SKU, name, description, warehouse_num) from m14.datawarehouse where warehouse_num=102")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


SKU=122, name=Samsung, description=Samsung laptop sales diminshed in our location, warehouse_num=102
SKU=126, name=samsung, description=Samsung galaxy s23 sales are peaked this year, warehouse_num=102
